In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
sess = tf.InteractiveSession()

In [4]:
#Fake digit produced by generator
INVALID = 10

In [5]:
def generator(z, is_train=True, out_channel_dim=1):
    alpha = 0.2
    fc_size = 512
    
    assert fc_size % 4 == 0
    
    with tf.variable_scope('generator', reuse=False if is_train==True else True):
        fc_1 = tf.layers.dense(z, 2*2*fc_size)
        
        deconv_2 = tf.reshape(fc_1, [-1, 2, 2, fc_size])
        batch_norm_2 = tf.layers.batch_normalization(deconv_2, training=is_train)
        lrelu_2 = tf.maximum(alpha * batch_norm_2, batch_norm_2)
        
        deconv_3 = tf.layers.conv2d_transpose(lrelu_2, fc_size // 2, 5, 2, padding="VALID")
        batch_norm_3 = tf.layers.batch_normalization(deconv_3, training=is_train)
        lrelu_3 = tf.maximum(alpha * batch_norm_3, batch_norm_3)
        
        deconv_4 = tf.layers.conv2d_transpose(lrelu_3, fc_size // 4, 5, 2, padding="SAME")
        batch_norm_4 = tf.layers.batch_normalization(deconv_4, training=is_train)
        lrelu_4 = tf.maximum(alpha * batch_norm_4, batch_norm_4)
        
        logits = tf.layers.conv2d_transpose(lrelu_4, out_channel_dim, 5, 2, padding="SAME")
        
        return tf.sigmoid(logits)

In [6]:
def discrim(images, reuse=False, is_train=True, num_channels=1):
    alpha = 0.2
    
    
    #I have no idea why the variable naming convention suddenly changed here
    with tf.variable_scope("discrim", reuse=reuse):
        conv1 = tf.layers.conv2d(images, 64, 5, 2, "SAME")
        lrelu1 = tf.nn.leaky_relu(conv1, alpha)
        
        conv2 = tf.layers.conv2d(lrelu1, 128, 5, 2, "SAME")
        batch_norm2 = tf.layers.batch_normalization(conv2, training=is_train)
        lrelu2 = tf.nn.leaky_relu(batch_norm2, alpha)
        
        conv3 = tf.layers.conv2d(lrelu2, 256, 5, 1, "SAME")
        batch_norm3 = tf.layers.batch_normalization(conv3, training=is_train)
        lrelu3 = tf.nn.leaky_relu(batch_norm3, alpha)
        
        flat_lrelu3 = tf.reshape(lrelu3, [-1, 7*7*256])
        
        logits = tf.layers.dense(flat_lrelu3, 11)
        
        return logits, tf.sigmoid(logits)  
        

In [7]:
def model_loss(input_real, input_labels, input_z, label_smoothing, num_channels=1):
    
    fake_out = generator(input_z)
    G_sample_image = tf.summary.image("Deep_GAN", fake_out, max_outputs=8)
    
    real_logits, real_probs = discrim(tf.reshape(input_real, [-1, 28, 28, num_channels]))
    fake_logits, fake_probs = discrim(fake_out, True)
    
    padded_labels = tf.pad(input_labels, [[0,0],[0,1]])
    
    D_loss_real = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=real_logits, labels=padded_labels))
    
    
    D_fake_labels = tf.pad(tf.zeros([tf.shape(fake_logits)[0], 10]), [[0,0],[0,1]], constant_values=1)
    D_loss_fake = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=fake_logits, labels=smooth(label_smoothing, D_fake_labels)))
    
    D_loss = D_loss_real + D_loss_fake
    
    fake_labels = tf.pad(tf.zeros([tf.shape(fake_logits)[0], 10]), [[0,0],[1, 0]], constant_values=1)
    G_loss = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=fake_logits, labels=smooth(label_smoothing, fake_labels)))
    
    return D_loss, G_loss, G_sample_image

def smooth(label_smoothing, tensor):
    label_base = 1 - label_smoothing
    
    return label_base * tensor
    

In [8]:
def opt(D_loss, G_loss, learning_rate, beta1):
    t_vars = tf.trainable_variables()
    
    d_train = tf.train.AdamOptimizer(learning_rate, beta1=beta1).minimize(
        D_loss, var_list=[var for var in t_vars if var.name.startswith("discrim")])
    g_train = tf.train.AdamOptimizer(learning_rate, beta1=beta1).minimize(
        G_loss, var_list=[var for var in t_vars if var.name.startswith("generator")])
    
    return d_train, g_train

In [9]:
Z = tf.placeholder(tf.float32, [None, 100])
X = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

label_smoothing = tf.placeholder_with_default(tf.constant(0, dtype=tf.float32), [])

D_loss, G_loss, G_sample_image = model_loss(X, y, Z, label_smoothing)
D_trainer, G_trainer = opt(D_loss, G_loss, 2e-4, 0.5)

In [10]:
def sample_Z(batch_size):
    return np.random.uniform(-1, 1, [batch_size, 100])

np.random.seed(0xDEADBEEF)

In [11]:
tf.global_variables_initializer().run()

summary_writer = tf.summary.FileWriter("tmp/logs", sess.graph)

In [14]:
BATCH_SIZE = 20
FAKE_BATCH_SIZE = 20
DISCR_RUNS = 100
GEN_RUNS = 100

for ITER in range(100000):
    batch = mnist.train.next_batch(BATCH_SIZE)
    fake_batch = sample_Z(FAKE_BATCH_SIZE)
    
    for _ in range(DISCR_RUNS):
        D_trainer.run(feed_dict={Z: fake_batch, X: batch[0], y: batch[1], label_smoothing: 0.1})
    
    for _ in range(GEN_RUNS):
        G_trainer.run(feed_dict={Z: fake_batch, X: batch[0], y: batch[1], label_smoothing: 0.1})
    
    if ITER % 20 == 0:
        if ITER % 50 == 0:
            D_train_loss = D_loss.eval(feed_dict={Z: fake_batch, X: batch[0], y: batch[1], label_smoothing: 0.1})
            G_train_loss = G_loss.eval(feed_dict={Z: fake_batch, X: batch[0], y: batch[1], label_smoothing: 0.1})

            print("Iter %05d: Losses: %.2f and %.2f"%(ITER, D_train_loss, G_train_loss))
        
        summary_writer.add_summary(G_sample_image.eval(feed_dict={Z: fake_batch, X: batch[0], y: batch[1]}), ITER)
        summary_writer.flush()

Iter 00000: Losses: 0.12 and 1.12


KeyboardInterrupt: 